In [1]:
# import GCP Demo 1 package
from gcpdemo1 import gcp, etl, data, tune, train, predict

In [2]:
# GCP variables
gcp_vars = {
    'credentials_path': '../credentials/ml-sandbox-1-191918-4714b5fd6e92.json', # local path to gcp credentials
    'project_name': 'ml-sandbox-1-191918', # gcp project name
    'bucket': 'gcp-cert-demo-1', # gcp bucket name
    'gcs_trainer_path': 'mlp_trainer_notebook_test/trainer-0.1.tar.gz', # path to write trainer package to within bucket
    'local_trainer_path': '../mlp_trainer', # local path to trainer package
}

In [3]:
# get GCP credentials
gcp_credentials = gcp.get_credentials(gcp_vars['credentials_path'])

In [ ]:
# build and upload trainer package
# TODO: does not work on windows because of no bash command
gcpdemo1.gcp.build_and_upload_trainer_package(
    bucket_name=gcp_vars['bucket'],
    destination_blob_name=gcp_vars['gcs_trainer_path'],
    local_trainer_package_path=gcp_vars['local_trainer_path'],
    credentials=gcp_vars['gcp_credentials']
)

In [ ]:
# TODO: tune here

In [4]:
# input training parameters
training_params = {
    'dense_neurons_1': 64,
    'dense_neurons_2': 32,
    'dense_neurons_3': 8,
    'activation': 'relu',
    'dropout_rate_1': 0.1,
    'dropout_rate_2': 0.1,
    'dropout_rate_3': 0.1,
    'optimizer': 'adam',
    'learning_rate': 0.1,
    'chunk_size': 500000,
    'batch_size': 1024,
    'epochs': 3,
    'validation_freq': 5,
    'kernel_initial_1': 'normal',
    'kernel_initial_2': 'normal',
    'kernel_initial_3': 'normal',
}

# input job parameters
job_params = {
    'job_id': 'mlp_trainer_notebook_test', # training job ID
    'job_dir': 'mlp_model_notebook_test' # model output directory
}

In [ ]:
# create mlp trainer
mlp_trainer = train.MLPTrainer(
    credentials=gcp_credentials,
    project_name=gcp_vars['project_name'],
    bucket=gcp_vars['bucket'],
    table_id='finaltaxi_encoded_sampled_small' # TODO: temp use of smaller table for testing
)

# begin training job
mlp_trainer.train(
    job_id=job_params['job_id'], 
    job_dir=job_params['job_dir'],
    package_uri='gs://gcp-cert-demo-1/mlp_trainer_src_test_9/27c3f0ffda0efc75d163b1693b0058d4d3f74e8f1eeb9251ab80037b14a82f63/trainer-0.1.tar.gz', # TODO: temp use until build fixed
    dense_neurons_1=training_params['dense_neurons_1'], 
    dense_neurons_2=training_params['dense_neurons_2'], 
    dense_neurons_3=training_params['dense_neurons_3'], 
    activation=training_params['activation'],
    dropout_rate_1=training_params['dropout_rate_1'], 
    dropout_rate_2=training_params['dropout_rate_2'], 
    dropout_rate_3=training_params['dropout_rate_3'], 
    optimizer=training_params['optimizer'], 
    learning_rate=training_params['learning_rate'], 
    chunk_size=training_params['chunk_size'], 
    batch_size=training_params['batch_size'], 
    epochs=training_params['epochs'],
    validation_freq=training_params['validation_freq'], 
    kernel_initial_1=training_params['kernel_initial_1'], 
    kernel_initial_2=training_params['kernel_initial_2'], 
    kernel_initial_3=training_params['kernel_initial_3'],
)

In [5]:
mlp_trainer = train.MLPTrainer(
    credentials=gcp_credentials,
    project_name=gcp_vars['project_name'],
    bucket=gcp_vars['bucket'],
    table_id='finaltaxi_encoded_sampled_small' # TODO: temp use of smaller table for testing
)
mlp_trainer.job_id = 'mlp_trainer_notebook_test_1573154941'
mlp_trainer.model_dir = 'mlp_model_notebook_test'

# check status of training job
mlp_trainer.training_status()

2019-11-07 15:17:35,848 [INFO]  Fetching status of training job "mlp_trainer_notebook_test_1573154941"
2019-11-07 15:17:35,850 [INFO]  URL being requested: GET https://www.googleapis.com/discovery/v1/apis/ml/v1/rest
2019-11-07 15:17:36,151 [INFO]  URL being requested: GET https://ml.googleapis.com/v1/projects/ml-sandbox-1-191918/jobs/mlp_trainer_notebook_test_1573154941?alt=json
2019-11-07 15:17:36,550 [INFO]  Training status response:


{'createTime': '2019-11-07T19:29:01Z',
 'endTime': '2019-11-07T19:39:12Z',
 'etag': 'Lgi7gmcNBiU=',
 'jobId': 'mlp_trainer_notebook_test_1573154941',
 'startTime': '2019-11-07T19:33:38Z',
 'state': 'SUCCEEDED',
 'trainingInput': {'args': ['--table-id',
                            'finaltaxi_encoded_sampled_small',
                            '--dense-neurons-1',
                            '64',
                            '--dense-neurons-2',
                            '32',
                            '--dense-neurons-3',
                            '8',
                            '--activation',
                            'relu',
                            '--dropout-rate-1',
                            '0.1',
                            '--dropout-rate-2',
                            '0.1',
                            '--dropout-rate-3',
                            '0.1',
                            '--optimizer',
                            'adam',
                            

In [6]:
# deploy trained model
mlp_trainer.deploy('mlp_model')

2019-11-07 15:17:48,435 [INFO]  Deploying model "mlp_model_1573157868" version "v_1573157771" from "gs://gcp-cert-demo-1/mlp_model_notebook_test"
2019-11-07 15:17:48,435 [INFO]  URL being requested: GET https://www.googleapis.com/discovery/v1/apis/ml/v1/rest
2019-11-07 15:17:48,751 [INFO]  URL being requested: POST https://ml.googleapis.com/v1/projects/ml-sandbox-1-191918/models?alt=json
2019-11-07 15:17:49,608 [INFO]  Model creation response:
2019-11-07 15:17:49,625 [INFO]  URL being requested: POST https://ml.googleapis.com/v1/projects/ml-sandbox-1-191918/models/mlp_model_1573157868/versions?alt=json


{'etag': 'WmEZgzNi3LU=',
 'name': 'projects/ml-sandbox-1-191918/models/mlp_model_1573157868',
 'regions': ['us-central1']}


2019-11-07 15:17:51,099 [INFO]  Version creation response:


{'metadata': {'@type': 'type.googleapis.com/google.cloud.ml.v1.OperationMetadata',
              'createTime': '2019-11-07T20:17:50Z',
              'modelName': 'projects/ml-sandbox-1-191918/models/mlp_model_1573157868',
              'operationType': 'CREATE_VERSION',
              'version': {'createTime': '2019-11-07T20:17:49Z',
                          'deploymentUri': 'gs://gcp-cert-demo-1/mlp_model_notebook_test',
                          'etag': 'dqQXimO23/M=',
                          'framework': 'TENSORFLOW',
                          'machineType': 'mls1-c1-m2',
                          'name': 'projects/ml-sandbox-1-191918/models/mlp_model_1573157868/versions/v_1573157771',
                          'pythonVersion': '3.5',
                          'runtimeVersion': '1.14'}},
 'name': 'projects/ml-sandbox-1-191918/operations/create_mlp_model_1573157868_v_1573157771-1573157869703'}
